In [2]:
import keras
from keras.layers import Input, Conv2D, Flatten, MaxPool2D, Dropout, Dense
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter
from keras import models
from keras.models import Model
from imgaug import augmenters
from random import randint
import pandas as pd
import numpy as np
import cv2
import time

# Problem 1

Choose a small (< 3,000) image dataset for classification. Include the link where you have downloaded the pictures from.

## 1

Train a model from scratch using what little data you have without any regularization, to set a baseline for what can be achieved.

DOWNLOAD HERE:

https://www.kaggle.com/zalando-research/fashionmnist/version/4

---


In [3]:
epochs = 30
batch_size = 256
verbose = 1

In [4]:
train = pd.read_csv("fashion-mnist_train.csv")
test = pd.read_csv("fashion-mnist_test.csv")
trainX = train[list(train.columns)[1:]].values
trainY = to_categorical(train['label'].values)
testX = train[list(train.columns)[1:]].values
testY = to_categorical(train['label'].values)

*Train-Validate Split*

In [5]:
trainX, valX, trainY, valY = train_test_split(
    trainX,
    trainY, 
    test_size=0.2,
    random_state=18
)

*Normalization*

In [6]:
trainX = trainX/255.
testX = testX/255.
valX = valX/255.

*Reshape*

In [7]:
trainX = trainX.reshape(-1, 28, 28, 1)
testX = testX.reshape(-1, 28, 28, 1)
valX = valX.reshape(-1, 28, 28, 1)

*Model*

In [8]:
# input
input_layer = Input(shape=(28, 28, 1))

# conv layers
conv_layer1   = Conv2D(32, (3, 3), activation='relu', padding='same')(input_layer)
conv_layer1   = MaxPool2D( (2, 2), padding='same')(conv_layer1)
conv_layer1   = Dropout(0.25)(conv_layer1)

conv_layer2   = Conv2D(64, (3, 3), activation='relu')(conv_layer1)
conv_layer2   = MaxPool2D( (2, 2), padding='same')(conv_layer2)
conv_layer2   = Dropout(0.25)(conv_layer2)

conv_layer3   = Conv2D(128, (3, 3), activation='relu')(conv_layer2)
conv_layer3   = Dropout(0.4)(conv_layer3)

# flatten and dense layers
flatten_layer = Flatten()(conv_layer3)
dense_layer   = Dense(128, activation='relu')(flatten_layer)
dense_layer   = Dropout(0.3)(dense_layer)

# output
output_layer  = Dense(10, activation='softmax')(dense_layer)

model = Model(input_layer, output_layer)
model.compile(
    optimizer=keras.optimizers.Adam(), 
    loss=keras.losses.categorical_crossentropy,
    metrics=['accuracy'])
if verbose==1:
    print(model.summary())

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 6, 6, 64)          0   

*Training*

In [9]:
tic = time.clock()

early_stopping = EarlyStopping(
    monitor='val_loss', 
    min_delta=0, 
    patience=10, 
    verbose=verbose, 
    mode='auto')

history = model.fit(
    trainX, 
    trainY, 
    epochs=epochs, 
    batch_size=batch_size, 
    verbose=verbose,
    validation_data=(valX, valY), 
    callbacks=[early_stopping])

toc = time.clock()
print(f"Conv Model took {toc-tic} seconds")

score = model.evaluate(testX, testY, verbose=0)
print('Test loss:    ', score[0])
print('Test accuracy:', score[1])
predictions = model.predict(testX)

Train on 48000 samples, validate on 12000 samples
Epoch 1/30
48000/48000 [==============================] - 50s 1ms/step - loss: 0.7914 - acc: 0.6999 - val_loss: 0.4500 - val_acc: 0.8372
Epoch 2/30
48000/48000 [==============================] - 49s 1ms/step - loss: 0.4767 - acc: 0.8246 - val_loss: 0.3729 - val_acc: 0.8661
Epoch 3/30
 5888/48000 [==>...........................] - ETA: 33s - loss: 0.4016 - acc: 0.8482

KeyboardInterrupt: 

*Plot*

In [ ]:
accuracy = history.history['acc']
val_accuracy = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(accuracy))

fig, ax = plt.subplots()
plt.plot(epochs, accuracy, 'bo', label='Training Accuracy')
plt.plot(epochs, val_accuracy, 'b', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.legend()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.tick_params(bottom=False, left=False)
frame1 = plt.gca()
fig.canvas.draw()
labels = [item.get_text() for item in ax.get_yticklabels()]
labels = list(map(float, labels))
labels = 100*np.array(labels)
labels = list(map(int, labels))
labels = list(map(str, labels))
labels = [label + "%" for label in labels]
ax.set_yticklabels(labels)
plt.savefig("Conv/ConvModelAccuracy.svg", format='svg')

fig, ax = plt.subplots()
plt.plot(epochs, loss, 'bo', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.legend()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.tick_params(bottom=False, left=False)
frame1 = plt.gca()
# frame1.axes.yaxis.set_ticklabels([])
plt.savefig("Conv/ConvModelLoss.svg", format='svg')